<a href="https://colab.research.google.com/github/lesya7maltseva/pure-Python-/blob/main/%D0%A0%D0%B0%D1%81%D1%87%D0%B5%D1%82_%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA(retention%2Cchurn%2C_LT%2C_MAU%2C_DAU%2C_WAU)_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%B4%D0%B0%D1%82%D0%BE%D0%B9_%D0%B8_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%B5%D0%BC_%D0%B8_%D1%81%D1%87%D0%B8%D1%82%D1%8B%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_csv_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-01-11 11:47:38--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-01-11 11:47:38--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv’

registrations.csv   100%[===================>]  14.57K  --.-KB/s    in 0s      

2025-01-11 11:47:38 (106 MB

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# Ваше решение
import csv
from datetime import datetime
# считываем даты регистрации юзеров, делаем словарь
dict_reg = {}

with open('registrations.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if datetime.strptime(row['registration_date'],'%Y-%m-%d').date() <= datetime.strptime('2021-01-31','%Y-%m-%d').date():
      dict_reg[row['user_id']] = row['registration_date']
    else: continue

# считываем входы, собираем список словарей
lst_entr = []
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if row['user_id'] in dict_reg.keys():
      lst_entr.append(row)
    else: continue

# добавляем реистрации ко входам:
for el in lst_entr:
  if el['user_id'] in dict_reg.keys():
    el['reg_date'] = dict_reg[el['user_id']]
  else: continue

# преобразуем в дату-время

for el in lst_entr:
    el['dif'] = (datetime.strptime(el['entry_date'], '%Y-%m-%d').date() - datetime.strptime(el['reg_date'], '%Y-%m-%d').date()).days

# создаем словарь где ключ - это разница в днях, а значение это id пользователя
dict_raw = {}

for el in lst_entr:
    if el['dif'] not in dict_raw.keys():
      dict_raw[el['dif']] = [el['user_id']]
    else: dict_raw[el['dif']].append(el['user_id'])

# создаем словарь где ключ - это разница в днях, а значение это количество уникальных пользователей
dict_retention = {}
for key, val in dict_raw.items():
  dict_retention[key] = len(set(val))

retention_15_day = round((dict_retention[15]/dict_retention[0] * 100),5)

### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# Ваше решение
import csv
from datetime import datetime
# считываем даты регистрации юзеров, делаем словарь
dict_reg = {}

with open('registrations.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if datetime.strptime(row['registration_date'],'%Y-%m-%d').date() <= datetime.strptime('2021-01-31','%Y-%m-%d').date():
      dict_reg[row['user_id']] = row['registration_date']
    else: continue

# считываем входы, собираем список словарей
lst_entr = []
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if row['user_id'] in dict_reg.keys():
      lst_entr.append(row)
    else: continue

# добавляем реистрации ко входам:
for el in lst_entr:
  if el['user_id'] in dict_reg.keys():
    el['reg_date'] = dict_reg[el['user_id']]
  else: continue

# преобразуем в дату-время

for el in lst_entr:
    el['dif'] = (datetime.strptime(el['entry_date'], '%Y-%m-%d').date() - datetime.strptime(el['reg_date'], '%Y-%m-%d').date()).days

# создаем словарь где ключ - это разница в днях, а значение это id пользователя
dict_raw = {}

for el in lst_entr:
    if el['dif'] not in dict_raw.keys():
      dict_raw[el['dif']] = [el['user_id']]
    else: dict_raw[el['dif']].append(el['user_id'])

# создаем словарь где ключ - это разница в днях, а значение это количество уникальных пользователей
dict_retention = {}
for key, val in dict_raw.items():
  dict_retention[key] = len(set(val))

rolling_retention = round((dict_retention[30]/dict_retention[0] * 100),5)

### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# Ваше решение
dict_reg = {}

with open('registrations.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    dict_reg[row['user_id']] = row['registration_date']

# считываем входы, собираем список словарей
lst_entr = []
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if row['user_id'] in dict_reg.keys():
      lst_entr.append(row)
    else: continue

# добавляем реистрации ко входам:
for el in lst_entr:
  if el['user_id'] in dict_reg.keys():
    el['reg_date'] = dict_reg[el['user_id']]
  else: continue

# преобразуем в дату-время

for el in lst_entr:
    el['dif'] = (datetime.strptime(el['entry_date'], '%Y-%m-%d').date() - datetime.strptime(el['reg_date'], '%Y-%m-%d').date()).days

# создаем словарь где ключ - это разница в днях, а значение это id пользователя
dict_raw = {}

for el in lst_entr:
    if el['dif'] not in dict_raw.keys():
      dict_raw[el['dif']] = [el['user_id']]
    else: dict_raw[el['dif']].append(el['user_id'])

# создаем словарь где ключ - это разница в днях, а значение это количество уникальных пользователей
dict_retention = {}
for key, val in dict_raw.items():
  dict_retention[key] = len(set(val))

lifetime = 0
for el in dict_retention.values():
  lifetime += el/1000
lifetime = round(lifetime,5)

### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`

In [ ]:
# Ваше решение
import csv
from datetime import datetime
# считываем даты регистрации юзеров, делаем словарь
dict_reg = {}

with open('registrations.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    dict_reg[row['user_id']] = row['registration_date']

# считываем входы, собираем список словарей
lst_entr = []
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if row['user_id'] in dict_reg.keys():
      lst_entr.append(row)
    else: continue

# добавляем реистрации ко входам:
for el in lst_entr:
  if el['user_id'] in dict_reg.keys():
    el['reg_date'] = dict_reg[el['user_id']]
  else: continue

# преобразуем в дату-время

for el in lst_entr:
    el['dif'] = (datetime.strptime(el['entry_date'], '%Y-%m-%d').date() - datetime.strptime(el['reg_date'], '%Y-%m-%d').date()).days

# выбираем всех пользователей которые зашли в 29й день и позже:
dict_curn = {29:[]}

for el in lst_entr:
    if el['dif'] >= 29:
      dict_curn[29].append(el['user_id'])

dict_curn[29] = len(set(dict_curn[29]))

churn_29 = 1 - dict_curn[29]/1000

### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [ ]:
# Ваше решение
lst_entr = []
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if datetime.strptime(row['entry_date'], '%Y-%m-%d').date() >= datetime.strptime('2021-12-01','%Y-%m-%d').date() and datetime.strptime(row['entry_date'], '%Y-%m-%d').date() <= datetime.strptime('2021-12-31','%Y-%m-%d').date():
      lst_entr.append(row)

lst_users = []
for el in lst_entr:
  lst_users.append(el['user_id'])

dec_mau = len(set(lst_users))

In [ ]:
# Ваше решение
lst_users_wau = []
for el in lst_entr:
  if datetime.strptime(el['entry_date'], '%Y-%m-%d').date() >= datetime.strptime('2021-12-25','%Y-%m-%d').date() and datetime.strptime(el['entry_date'], '%Y-%m-%d').date() <= datetime.strptime('2021-12-31','%Y-%m-%d').date():
    lst_users_wau.append(el['user_id'])

dec_wau = len(set(lst_users_wau))

In [ ]:
# Ваше решение
lst_users_dau = []
for el in lst_entr:
  if datetime.strptime(el['entry_date'], '%Y-%m-%d').date() == datetime.strptime('2021-12-31','%Y-%m-%d').date() :
    lst_users_dau.append(el['user_id'])

dec_dau = len(set(lst_users_dau))

### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [ ]:
# Ваше решение
dict_raw_mau = {}
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if row['entry_date'][:-3] in dict_raw_mau.keys():
      dict_raw_mau[row['entry_date'][:-3]].append(row['user_id'])
    else : dict_raw_mau[row['entry_date'][:-3]] = [row['user_id']]

dict_avg_mau = {}
for key, val in dict_raw_mau.items():
  dict_avg_mau[key] = len(set(val))
avg_mau = round(sum(dict_avg_mau.values())/len(dict_avg_mau.values()),5)


In [ ]:
# Ваше решение
lst_wau = []
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    row['week'] = datetime.strptime(row['entry_date'], '%Y-%m-%d').isocalendar().week
    lst_wau.append(row)

dict_raw_wau = {}
for el in lst_wau:
  if el['week'] in dict_raw_wau.keys():
      dict_raw_wau[el['week']].append(el['user_id'])
  else : dict_raw_wau[el['week']] = [el['user_id']]

dict_avg_wau = {}
for key, val in dict_raw_wau.items():
  dict_avg_wau[key] = len(set(val))
dict_avg_wau

avg_wau = round(sum(dict_avg_wau.values())/len(dict_avg_wau.values()),5)
avg_wau

89.86792

In [ ]:
# Ваше решение
dict_raw_dau = {}
with open('entries.csv') as f:
  reader = csv.DictReader(f, delimiter=';')
  for row in reader:
    if row['entry_date'] in dict_raw_dau.keys():
      dict_raw_dau[row['entry_date']].append(row['user_id'])
    else : dict_raw_dau[row['entry_date']] = [row['user_id']]

dict_avg_dau = {}
for key, val in dict_raw_dau.items():
  dict_avg_dau[key] = len(set(val))
avg_dau = round(sum(dict_avg_dau.values())/len(dict_avg_dau.values()),5)

avg_dau

40.5589